# VOLTTRON Hackathon 2 Notebook

This notebook sets up and runs a simulation. Most of the notebook's setup and execution is done with shell commands, called from Python.

# Setup: Prepare the Volttron Environment

VOLTTRON must be installed before using this notebook. For detailed instructions on
installing and configuring a VOLTTRON/Jupyter server environment, 
see [Jupyter Notebooks](http://volttron.readthedocs.io/en/devguides/supporting/utilities/JupyterNotebooks.html) 
in VOLTTRON ReadTheDocs.

As is described in that guide, environment variables should have been defined before starting 
the Jupyter server:

````
$ export VOLTTRON_ROOT=~/repos/volttron
````
        (path of the VOLTTRON repository, installed prior to running bootstrap)

````
$ export VOLTTRON_HOME=~/.volttron
````
        (directory in which the VOLTTRON instance runs)

The first VOLTTRON instance on a server usually runs, by convention, in ~/.volttron.
If multiple VOLTTRON instances are to be run on a single host, each must have its own VOLTTRON_HOME.

Also before starting the Jupyter server, a VOLTTRON virtual environment should have been 
activated by executing the following in $VOLTTRON_ROOT:

````
$ source env/bin/activate
````

The simulation software resides in a separate repository, volttron-applications.

It must be downloaded from github, creating a new directory parallel to $VOLTTRON_ROOT, as follows:

````
$ cd $VOLTTRON_ROOT
$ cd ..
$ git clone git://github.com/VOLTTRON/volttron-applications.git
````

Then a symbolic link to it, named "applications", should be added under $VOLTTRON_ROOT:

````
$ cd $VOLTTRON_ROOT
$ ln -s ../volttron-applications/ applications
````

The Python code below does some initialization to prepare for the steps that follow.

In [ ]:
import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
approot = vroot + '/applications'
notebooks = vroot + '/examples/JupyterNotebooks'
vhome = %env VOLTTRON_HOME
data_dir = vhome + '/data'
conf = vhome + '/config'
with open(conf, 'rb') as config_file:
    web_address_line = [line for line in config_file.readlines() if 'bind-web-address' in line][0]
web_address = web_address_line.split(':')[-2][2:]
web_port = web_address_line.split(':')[-1].strip()

try:
    # The user home directory if running in a Docker container. Used when looking at the VOLTTRON log.
    volttron_user_home = %env VOLTTRON_USER_HOME
except:
    # When not running in a Docker container, VOLTTRON is managed from VOLTTRON_ROOT.
    volttron_user_home = %env VOLTTRON_ROOT
print('VOLTTRON_ROOT={}'.format(vroot))
print('VOLTTRON applications root={}'.format(approot))
print('Jupyter notebooks directory={}'.format(notebooks))
print('VOLTTRON_HOME={}'.format(vhome))
print('VOLTTRON data directory={}'.format(data_dir))
print('VOLTTRON_USER_HOME={}'.format(volttron_user_home))
print('Web Address={}'.format(web_address))
print('Web Port={}'.format(web_port))

# Enable one Jupyter notebook to import and run the contents of another notebook.
os.chdir(notebooks)
import notebook_loader

# Import notebooks containing utility functions.
import NotebookUtilities
import ConfigureAgents

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

print("Initialization complete")

# Setup: Shut Down All Agents and Delete Test Databases

This ensures a clean agent installation process by the notebook.

In [ ]:
print('Wait for the list to be displayed.')
print('Confirm that no agents are listed as running...\n')

# Shut down all agents.
NotebookUtilities.sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
NotebookUtilities.print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Delete the Historian's SQLite database.
db_path = data_dir + '/historian.sqlite'
if os.path.exists(db_path):
    NotebookUtilities.sh('rm {0}'.format(db_path))
print('Deleted {}'.format(db_path))

# Delete the Platform Historian's SQLite database.
db_path = data_dir + '/platform.historian.sqlite'
if os.path.exists(db_path):
    NotebookUtilities.sh('rm {0}'.format(db_path))
print('Deleted {}'.format(db_path))

# Setup: Configure the Simulation Device Drivers

In [ ]:
print('Wait for the simulation driver configs to be displayed.')
print('Then confirm that all of these configs appear in it...\n')

driver_root = approot + '/kisensum/Simulation/SimulationDriverAgent/'

# Install simload, the simulated load driver
NotebookUtilities.install_driver_registry_csv(store='simulation.driver', driver_root=driver_root, name='simload.csv', csv='simload.csv')
NotebookUtilities.install_driver_config(store='simulation.driver', driver_root=driver_root, name='devices/campus1/building1/unit1/simload', config='simload.config')

# Install simmeter, the simulated meter driver
NotebookUtilities.install_driver_registry_csv(store='simulation.driver', driver_root=driver_root, name='simmeter.csv', csv='simmeter.csv')
NotebookUtilities.install_driver_config(store='simulation.driver', driver_root=driver_root, name='devices/campus1/building1/unit1/simmeter', config='simmeter.config')

# Install simpv, the simulated PV driver
NotebookUtilities.install_driver_registry_csv(store='simulation.driver', driver_root=driver_root, name='simpv.csv', csv='simpv.csv')
NotebookUtilities.install_driver_config(store='simulation.driver', driver_root=driver_root, name='devices/campus1/building1/unit1/simpv', config='simpv.config')

# Install simstorage, the simulated storage driver
NotebookUtilities.install_driver_registry_csv(store='simulation.driver', driver_root=driver_root, name='simstorage.csv', csv='simstorage.csv')
NotebookUtilities.install_driver_config(store='simulation.driver', driver_root=driver_root, name='devices/campus1/building1/unit1/simstorage', config='simstorage.config')

# List the Simulation Driver configuration to confirm that the drivers were installed successfully.
NotebookUtilities.print_sh('volttron-ctl config list simulation.driver')

# Execution: Install and start SimulationDriverAgent

SimulationDriverAgent is a copy of MasterDriverAgent, revised to manage simulated
device drivers. An important difference is that the simulated execution speed of
SimulationDriverAgent is governed by a simulated clock, which can speed up or slow
down relative to real time.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=approot+'/kisensum/Simulation/SimulationDriverAgent',
                                id='simulation.driver',
                                config=approot+'/kisensum/Simulation/SimulationDriverAgent/simulationdriver.config')

# Execution: Install and start SimulationClockAgent

SimulatedClockAgent manages a simulation's clock. While a simulation is in progress,
it responds to "what time is it?" RPC calls by returning the current simulated time.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=approot+'/kisensum/Simulation/SimulationClockAgent',
                                id='simulationclock', 
                                config=approot+'/kisensum/Simulation/SimulationClockAgent/simulationclock.config')

# Execution: Install and start SimulationAgent

SimulationAgent is a control agent that manages a simulation. Based on
config parameters, it decides when to start and stop a simulation, how fast
the simulation's clock should move relative to real time, which simulated
devices will participate, and various adjustable parameters for each of the
simulated devices.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=approot+'/kisensum/Simulation/SimulationAgent',
                                id='simulationagent', 
                                config=ConfigureAgents.configure_simulation_agent())

# Execution: Install and start ActuatorAgent

An ActuatorAgent is a device's gatekeeper, fielding device-control requests
from other agents and ensuring that the requests don't conflict.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/ActuatorAgent', 
                                id='simulation.actuator', 
                                config=ConfigureAgents.configure_actuator_agent())

# Execution: Install and start ListenerAgent

ListenerAgent monitors the VOLTTRON message bus for messages matching its criteria.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/examples/ListenerAgent', 
                                id='listener', 
                                config=ConfigureAgents.configure_listener_agent())

# Execution: Install and start SQLHistorian

SQLHistorian monitors the VOLTTRON message bus for specific types of data
(often device telemetry), and writes the data to a SQLite database.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/SQLHistorian', 
                                id='sqlite_historian', 
                                config=ConfigureAgents.configure_sql_historian())

# Execution: Install and start Platform Historian

PlatformHistorian is a special version of SQLHistorian that writes device
telemetry to a SQLite database for display by the user interface that is
managed by VolttronCentral. 

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/SQLHistorian', 
                                id='platform.historian', 
                                config=ConfigureAgents.configure_platform_historian())

# Execution: Install and start VolttronCentralPlatform

VolttronCentralPlatform manages messaging among multiple VOLTTRON instances.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/VolttronCentralPlatform',
                                id='platform.agent',
                                config=ConfigureAgents.configure_volttron_central_platform())

# Execution: Install and start VolttronCentral

VolttronCentral manages VOLTTRON's web user interface.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/VolttronCentral',
                                id='volttron.central',
                                config=vroot+'/services/core/VolttronCentral/config')

# Execution: Check Agent Status

In [ ]:
print('Wait for the list to be displayed.')
print('Then confirm that each started agent is running...\n')

# List agent status to verify that the started agents have status "running".
NotebookUtilities.print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Tail the Log File

Display the last 100 lines of the VOLTTRON log file. Useful when debugging.

In [ ]:
NotebookUtilities.print_sh('tail -100 {}/log1'.format(volttron_user_home))

Display the end of the log file again.
This time, filter the output to display only lines logged by simulationagent.
(Note: this may return an error if none of the lines match the filter.)

In [ ]:
NotebookUtilities.print_sh('tail -2000 {}/log1 | grep simulationagent'.format(volttron_user_home))

# Data Reporting: Describe the Historian's Database Schema

Start the data reporting process by displaying the schema of the Historian's SQLite database.

In [ ]:
NotebookUtilities.run_sqlite_cmd(data_dir + '/historian.sqlite', '".schema"')

# Data Reporting: List the Topics

List each topic in the database's topics_table. This is the list of each type of data that has been captured and stored.

In [ ]:
NotebookUtilities.run_sqlite_cmd(data_dir + '/historian.sqlite', '"SELECT * FROM topics_table;"')

# Data Reporting: List Values for a Single Topic

Select a single topic by name, and list each value in the database for it.

In [ ]:
topic_name = 'campus1/building1/unit1/simstorage/soc_kwh'

display_variables = 'ts, value_string'
join_statement = 'INNER JOIN topics_table on (data_table.topic_id = topics_table.topic_id) '
sqlite_cmd = '''"SELECT {0} FROM data_table {1} WHERE topics_table.topic_name = '{2}';"'''.format(
    display_variables,
    join_statement,
    topic_name)
print('sqlite command: \n{0}\n'.format(sqlite_cmd))

NotebookUtilities.run_sqlite_cmd(data_dir + '/historian.sqlite', sqlite_cmd)

# Data Reporting: Graph Values for a Single Topic

Use numpy and matplotlib to produce a graph of the values for a topic.

In [ ]:
topic_name = 'campus1/building1/unit1/simstorage/soc_kwh'

display_variables = 'ts, value_string'
join_statement = 'INNER JOIN topics_table on (data_table.topic_id = topics_table.topic_id) '
sqlite_cmd = '''SELECT {0} FROM data_table {1} WHERE topics_table.topic_name = '{2}';'''.format(
    display_variables,
    join_statement,
    topic_name)
print('sqlite command: \n{0}\n'.format(sqlite_cmd))

import numpy
import matplotlib.pyplot as plt
from matplotlib import dates

# Connect to the SQLite database
conn = sqlite3.connect(data_dir + '/historian.sqlite')
c = conn.cursor()

# Populate graphArray with the result of querying the database for the specified topic.
graphArray = []
for row in c.execute(sqlite_cmd):
    # Remove parentheses and single quotes
    row_string_filtered = str(row).translate(None, "()'u\'")
    # In Python 3, the Unicode string would need to be filtered like this:
    # row_string_filtered = str(row).translate({ord(c): None for c in "()'u\'"})
    graphArray.append(row_string_filtered)

if graphArray:
    timestamps, values = numpy.loadtxt(graphArray,
                                       delimiter=',',
                                       unpack=True,
                                       converters={0: dates.strpdate2num('%Y-%m-%dT%H:%M:%S.%f+00:00')})
    fig = plt.figure()
    fig.add_subplot(1, 1, 1, facecolor='white')
    plt.plot_date(x=timestamps, y=values, fmt='b-')
    plt.gcf().autofmt_xdate()
    plt.show()
else:
    print('No data returned from query')

# STOP HERE. We will break for lunch and do the exercises when we get back

# Exercise #1: Changing the Simulation Storage Algorithm

What happens if we want to change how our control algorithm works?

Now let's try to keep our microgrid power below a threshold, but using fifteen minute averages. Your implemented algorithm should send setpoints that will control the microgrid such that at the end of fifteem minute intervals (based on a simulated clock, this will be on the hour, fifteen after, half-past, and quarter till), the average power for that interval does not exceed a set threshold.

After loading the stubbed calculate_setpoint function, write a python function that returns a different function

In [ ]:
algorithm_file = approot+'/kisensum/Simulation/SimulationAgent/simulation/custom_setpoint.py'
%load $algorithm_file

After you think you have a solution, copy the code from the above cell, and insert it below the %%writefile $file_name line in the following code block. This will write the code to our agent files.

In [ ]:
%%writefile $algorithm_file

Now let's rerun our SimulationAgent and take a look at the results.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=approot+'/kisensum/Simulation/SimulationAgent',
                                id='simulationagent', 
                                config=ConfigureAgents.configure_simulation_agent_with_custom_rule())

We can look at our new simulation in two places:
1. Trace output in the VOLTTRON log file
2. The graph that we created earlier today (see "Graph Values for a Single Topic")

# Exercise #2: Dynamically update the Meter Target

So far, we've had a static target for our algorithm. What if we want to optimize our algorithm so that once we set a peak, that becomes our new control point?

Lets take a look at the calculated average power at the end of each 15 minute interval. If it sets a new peak, we can update our target and control to the new peak. Bonus: The peak is usually set on a monthly basis. Bonus points if on a new month, we roll the established peak back to a new, low value.

# Exercise #3: Use OpenADR to control our threshold target

OpenADR is a Demand Response (DR) protocol that uses events to indicate when periods of load shed should occur. There are two major components to OpenADR: the VTN (Virtual Top Node) - which is reponsible for defining DR events, and the VEN (Virtual End Node) - which is reponsible for receiving DR events. In VOLTTRON, we can create a VEN Agent, which will listen for VTN defined DR events and publish them to the VOLTTRON message bus.

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=vroot+'/services/core/OpenADRVenAgent',
                                id='openadrvenagent', 
                                config=ConfigureAgents.configure_ven_agent())

When the OpenADR Events get put on the message bus by the VEN agent, we need our Simulation Agent to pull the events off of the message bus and process them. Here's the code in our simulation agent that subscribes to the OPENADR_EVENT topic, and calls and processes the messages as they come through.

    @Core.receiver('onstart')
    def onstart_method(self, sender):
        """The agent has started. Perform initialization and spawn the main process loop."""
        _log.debug('Starting agent')

        # Subscribe to the VENAgent's event and report parameter publications.
        self.vip.pubsub.subscribe(peer='pubsub', prefix=topics.OPENADR_EVENT, callback=self.receive_event)

    def receive_event(self, peer, sender, bus, topic, headers, message):
        """(Subscription callback) Receive a list of active events as JSON."""
        debug_string = 'Received event: ID={}, status={}, start={}, end={}, opt_type={}, all params={}'
        _log.debug(debug_string.format(message['event_id'], message['status'], message['start_time'],
                                       message['end_time'], message['opt_type'], message))
        self.sim_data['start_event'] = message['start_time']
        self.sim_data['end_event'] = message['end_time']

Now we need to use this event data to set our meter target. Let's load up our stubbed method for calculating this.

In [ ]:
meter_target_file = approot+'/kisensum/Simulation/SimulationAgent/simulation/meter_target.py'
%load $meter_target_file

Now, rather than just returning the default_meter_target, check the simulation data for the presence of an event and return the appropriate meter target.

Don't forget to copy your code back to the file!

In [ ]:
%%writefile $meter_target_file

Now we can simulate a VTN agent and send an event to the VEN agent. Three parameters are required:

1. **event_id**: unique id. each event will need a new id
2. **start_delay**: how many seconds in the future (based on simulated clock time) the event will start
3. **event duration**: how many seconds the event will last

First though, let's start up a new simulation. After it starts, go ahead and send an event!

In [ ]:
print('Wait for the message that the agent has been started.')
NotebookUtilities.install_agent(dir=approot+'/kisensum/Simulation/SimulationAgent',
                                id='simulationagent', 
                                config=ConfigureAgents.configure_simulation_agent_with_custom_rule())

When ready, send an OpenADR event.

If you do this more than once (for example, while debugging), be sure to use a different event_id each time.

In [ ]:
event_id = 1
NotebookUtilities.send_vtn_event(vroot, web_address, web_port, event_id=event_id, start_delay=300, event_duration=3600)

# Shutdown: Stop all agents

When finished, stop all VOLTTRON agents.

In [ ]:
# Stop all agents.
NotebookUtilities.sh('volttron-ctl shutdown')

# Verify that all agents have been stopped.
NotebookUtilities.print_sh('volttron-ctl status', stderr=subprocess.STDOUT)